In [408]:
# #em teoria, isso faz tudo, mas não tem suporte para português
# from nemo_text_processing.text_normalization.normalize import Normalizer

# def normalize_text(input_text, language='de'):
#     normalizer = Normalizer(lang=language, input_case='cased')
#     normalized = normalizer.normalize(text=input_text, punct_post_process=True)

#     return normalized

# #print(normalize_text("Dr. Schmidt zahle 1.245Є für 2 Artikel um 15:10 Uhr.","de"))
# #print(normalize_text("Die Temperatur beträgt 22°C bei einer Luftfeuchtigkeit von 10% am 22. Mai 2024 um 15:13 Uhr.","de"))
# print(normalize_text("Dr. Smith paid $1,234 for 2 items at 3pm after waiting outside at 72°F on may, 15th, 2024. While waiting for the train to arrive at 15:45 he called a support hotline at 1-800-555-0123.","en"))

In [409]:
#serve para corrigir ortografia, mas as muitas vezes ele corrige palavras que não deveria pq não as conhece
# from spellchecker import SpellChecker 

# spell = SpellChecker(language='pt')

# misspelled = spell.unknown(['varios', 'palabra', 'letra'])

# for word in misspelled:
#     print(spell.correction(word))

In [410]:
from num2words import num2words
import re
from abreviacoes import abreviacoes

In [411]:
def limpar_simbolos(texto):
    texto = re.sub(r"http\S+", "", texto)  # Remove URLs
    texto = re.sub(r"[^\w\s,.!?áéíóúãõâêôçÁÉÍÓÚÃÕÂÊÔÇ]", "", texto)  # Remove emojis e símbolos
    return texto

In [412]:
def converter_moeda(texto):
    simbolos_moeda = {
        r'R\$': 'reais',
        r'USD': 'dólares',
        r'U\$': 'dólares'
    }

    for simbolo_regex, nome in simbolos_moeda.items():
        # captura o símbolo + número com possíveis pontos de milhar e opcional vírgula decimal
        padrao = fr'{simbolo_regex}\s*(\d{{1,3}}(?:\.\d{{3}})*(?:,\d+)?)'

        def substituir(m):
            orig = m.group(1)           # ex: "3.200,75" ou "10" ou "2.500"
            # se tiver vírgula, é decimal no formato BR
            if ',' in orig:
                # remove pontos de milhar, mantém vírgula
                inteiro_str, dec_str = orig.replace('.', '').split(',', 1)
                inteiro = int(inteiro_str)
                # garante dois dígitos nos centavos
                dec_str = (dec_str + "00")[:2]
                centavos = int(dec_str)

                partes = []
                if inteiro > 0:
                    partes.append(f"{num2words(inteiro, lang='pt_BR')} {nome}")
                if centavos > 0:
                    txt_cent = num2words(centavos, lang='pt_BR')
                    txt_cent += " centavo" + ("s" if centavos > 1 else "")
                    if inteiro > 0:
                        partes.append(f"e {txt_cent}")
                    else:
                        partes.append(txt_cent)
                return " ".join(partes)

            # se não tem vírgula mas tem ponto (milhar) — trata como inteiro
            elif '.' in orig:
                inteiro = int(orig.replace('.', ''))
                return f"{num2words(inteiro, lang='pt_BR')} {nome}"

            # caso puro inteiro (sem ponto nem vírgula)
            else:
                inteiro = int(orig)
                return f"{num2words(inteiro, lang='pt_BR')} {nome}"

        texto = re.sub(padrao, substituir, texto)

    return texto


In [413]:
def converter_numeros(texto):
    # 1. Casos de moeda com símbolos conhecidos
    texto = converter_moeda(texto)

    # 2. Números colados a unidades (ex: 10kg → 10 kg). Só coloca espaço.
    texto = re.sub(r'(\d+)(°?[a-zA-Z²³µ%]+)', r'\1 \2', texto)

    # 3. Números com vírgula ou ponto → "dois vírgula cinco"
    texto = re.sub(r'\b(\d+)[.,](\d+)\b', lambda m: f"{num2words(int(m.group(1)), lang='pt_BR')} vírgula {num2words(int(m.group(2)), lang='pt_BR')}", texto)

    # 4. Horários → "10:45 -> dez e quarenta e cinco"
    texto = re.sub(r'\b([0-1]?\d|2[0-3]):00h?\b', lambda m: num2words(int(m.group(1)), lang='pt_BR'), texto)
    texto = re.sub(r'\b(\d+)[:](\d+)\b', lambda m: f"{num2words(int(m.group(1)), lang='pt_BR')} e {num2words(int(m.group(2)), lang='pt_BR')}", texto)

    # 5. Números inteiros simples → por extenso
    texto = re.sub(r'\b\d+\b', lambda m: num2words(int(m.group()), lang='pt_BR'), texto)

    return texto

In [414]:
def expandir_abreviacoes(texto):
    for abrev, exp in abreviacoes.items():
        padrao = r'(?<!\w)' + re.escape(abrev) + r'(?!\w)'
        texto = re.sub(padrao, exp, texto)
    return texto


In [415]:
def normalizar_texto(texto):
    texto = converter_numeros(texto)
    texto = expandir_abreviacoes(texto)
    texto = limpar_simbolos(texto)
    return texto

In [416]:
testes = [
    # Moeda e valores
    "Ganhei USD 1.000,50 em um sorteio em MT.",  # Moeda, ponto e vírgula
    "Paguei USD 3.200,75 por tudo.",  # Moeda com vírgula e ponto
    "O preço é U$ 50,25 e vale a pena.",  # U$ com vírgula
    "R$ 10.000,00 foi o prêmio.",  # R$ com vírgula e ponto
    "Custou R$1.000,50",  # R$ sem espaço, com vírgula
    "USD1.500,10",  # USD junto com número com ponto e vírgula
    "O preço total é R$10.000",  # R$ com número simples
    # Horários
    "O rodízio vai até as 10:30 em dias úteis.",  # Horário com minutos
    "A reunião é às 14:00h e vai até 17:00.",  # Horário com 'h' no final
    "O voo parte às 9:00",  # Horário com 00 minutos
    # Temperaturas
    "A temperatura máxima será de 30°C.",  # Temperatura com símbolo
    "A previsão é de 35,5°C para hoje.",  # Temperatura com vírgula
    # Siglas de estados e unidades
    "A reunião será em SP, na Av. Paulista.",  # Siglas e abreviações
    "O evento ocorre na Rua 25 de Março, no centro de SP.",  # Rua e sigla
    "O tratamento é dado pelo Dr. João Silva.",  # Tratamento e abreviação
    "A viagem será por Av. Brasil, em RJ.",  # Avenida e sigla
    "O time joga em MG e depois vai para a BA.",  # Siglas de estados
    "A medida de altura é de 3m de altura.",  # Unidade de medida
    "Comprei 10kg de arroz e 5m de tecido.",  # Unidade de medida (peso e distância)
    "Ele correu 100m em 10s.",  # Unidade de medida (distância e tempo)
    "Visite nosso site: https://www.exemplo.com para mais informações! 😊",  # URL e emoji
    "Veja este artigo em https://www.meusite.com.br/novidades?utm_source=site",  # URL com parâmetros
    "Não perca nossas promoções! 💰🎉 http://promo.com.br/descontos-20",  # URL e emojis com símbolos
]



for i, texto in enumerate(testes):
    print("====================")
    print(f"Teste {i}")
    print(texto)
    print(normalizar_texto(texto))
    print("====================",end='\n\n')


Teste 0
Ganhei USD 1.000,50 em um sorteio em MT.
Ganhei mil dólares e cinquenta centavos em um sorteio em Mato Grosso.

Teste 1
Paguei USD 3.200,75 por tudo.
Paguei três mil e duzentos dólares e setenta e cinco centavos por tudo.

Teste 2
O preço é U$ 50,25 e vale a pena.
O preço é cinquenta dólares e vinte e cinco centavos e vale a pena.

Teste 3
R$ 10.000,00 foi o prêmio.
dez mil reais foi o prêmio.

Teste 4
Custou R$1.000,50
Custou mil reais e cinquenta centavos

Teste 5
USD1.500,10
mil e quinhentos dólares e dez centavos

Teste 6
O preço total é R$10.000
O preço total é dez mil reais

Teste 7
O rodízio vai até as 10:30 em dias úteis.
O rodízio vai até as dez e trinta em dias úteis.

Teste 8
A reunião é às 14:00h e vai até 17:00.
A reunião é às catorze horas e vai até dezessete.

Teste 9
O voo parte às 9:00
O voo parte às nove

Teste 10
A temperatura máxima será de 30°C.
A temperatura máxima será de trinta graus celsius.

Teste 11
A previsão é de 35,5°C para hoje.
A previsão é de tr